<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# NLP Basics

**LSM Programming Challenge**

&copy; Dr. Yves J. Hilpisch

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

## ChatGPT

### 4o

In [ ]:
!git clone https://github.com/tpq-classes/natural_language_processing.git
import sys
sys.path.append('natural_language_processing')


In [ ]:
import numpy as np
from scipy.stats import norm
import time

class BlackScholesMonteCarlo:
    """
    Class for valuing American and European options using:
    - American: Longstaff-Schwartz Least Squares Monte Carlo method
    - European: Closed-form Black-Scholes-Merton formulas
    """
    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps
        self.dt = T / steps
        self.discount = np.exp(-r * self.dt)

    def _validate_inputs(self):
        if any(param <= 0 for param in [self.S0, self.K, self.T, self.paths, self.steps]):
            raise ValueError("Inputs S0, K, T, paths, and steps must be positive.")
        if self.sigma < 0:
            raise ValueError("Volatility sigma must be non-negative.")

    def _simulate_paths(self):
        """Simulate asset price paths using geometric Brownian motion."""
        np.random.seed(0)  # For reproducibility
        increments = np.random.standard_normal((self.steps, self.paths))
        increments = (self.r - 0.5 * self.sigma ** 2) * self.dt + self.sigma * np.sqrt(self.dt) * increments
        paths = np.zeros((self.steps + 1, self.paths))
        paths[0] = self.S0
        for t in range(1, self.steps + 1):
            paths[t] = paths[t - 1] * np.exp(increments[t - 1])
        return paths

    def _lsm_american_option(self, option_type='put'):
        """Least Squares Monte Carlo for American option valuation."""
        self._validate_inputs()
        paths = self._simulate_paths()
        payoff = np.maximum(self.K - paths if option_type == 'put' else paths - self.K, 0)
        cashflow = payoff[-1]

        for t in range(self.steps - 1, 0, -1):
            discount_cashflow = cashflow * self.discount
            itm_indices = payoff[t] > 0

            if np.any(itm_indices):
                regression = np.polyfit(paths[t][itm_indices], discount_cashflow[itm_indices], 2)
                continuation_value = np.polyval(regression, paths[t][itm_indices])
                exercise_indices = payoff[t][itm_indices] > continuation_value
                cashflow[itm_indices] = np.where(exercise_indices, payoff[t][itm_indices], discount_cashflow[itm_indices])
            else:
                cashflow *= self.discount

        return np.exp(-self.r * self.dt) * np.mean(cashflow)

    def _european_option_closed_form(self, option_type='put'):
        """Closed-form Black-Scholes-Merton formula for European options."""
        self._validate_inputs()
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)

        if option_type == 'call':
            return self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            return self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)

    def value_american_put(self):
        """Value American put option using LSM method."""
        return self._lsm_american_option(option_type='put')

    def value_american_call(self):
        """Value American call option using LSM method."""
        return self._lsm_american_option(option_type='call')

    def value_european_put(self):
        """Value European put option using closed-form formula."""
        return self._european_option_closed_form(option_type='put')

    def value_european_call(self):
        """Value European call option using closed-form formula."""
        return self._european_option_closed_form(option_type='call')

# Test code with execution time measurement
if __name__ == "__main__":
    option_pricer = BlackScholesMonteCarlo()
    
    start_time = time.time()
    american_put_value = option_pricer.value_american_put()
    american_call_value = option_pricer.value_american_call()
    execution_time = time.time() - start_time

    european_put_value = option_pricer.value_european_put()
    european_call_value = option_pricer.value_european_call()

    print(f"American Put Value: {american_put_value:.3f} (Expected: 4.478)")
    print(f"American Call Value: {american_call_value:.3f}")
    print(f"European Put Value: {european_put_value:.3f} (Expected: 3.844)")
    print(f"European Call Value: {european_call_value:.3f}")
    print(f"Execution Time (American options): {execution_time:.3f} seconds")

### o3-mini-high

In [ ]:
import numpy as np
import math
import time


class OptionValuation:
    """
    A class for valuing European and American options in the Black–Scholes–Merton (1973) model.
    
    American options are priced via simulation using the Longstaff–Schwartz (2001) least squares Monte Carlo method.
    European options are priced in closed form using the Black–Scholes formula.
    
    Parameters
    ----------
    S0 : float, optional
        Initial asset price (default is 36).
    K : float, optional
        Strike price (default is 40).
    T : float, optional
        Time to maturity in years (default is 1.0).
    r : float, optional
        Risk-free interest rate (default is 0.06).
    sigma : float, optional
        Volatility of the underlying asset (default is 0.2).
    paths : int, optional
        Number of simulated paths (default is 100000).
    steps : int, optional
        Number of time steps in the simulation (default is 50).
    """

    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps

        self._validate_inputs()

    def _validate_inputs(self):
        """Validate input parameters."""
        if not isinstance(self.S0, (int, float)) or self.S0 <= 0:
            raise ValueError("S0 must be a positive number.")
        if not isinstance(self.K, (int, float)) or self.K <= 0:
            raise ValueError("K must be a positive number.")
        if not isinstance(self.T, (int, float)) or self.T <= 0:
            raise ValueError("T must be a positive number.")
        if not isinstance(self.r, (int, float)):
            raise ValueError("r must be a numeric value.")
        if not isinstance(self.sigma, (int, float)) or self.sigma <= 0:
            raise ValueError("sigma must be a positive number.")
        if not isinstance(self.paths, int) or self.paths < 1:
            raise ValueError("paths must be a positive integer.")
        if not isinstance(self.steps, int) or self.steps < 1:
            raise ValueError("steps must be a positive integer.")

    @staticmethod
    def _norm_cdf(x):
        """
        Compute the cumulative distribution function for a standard normal variable.
        
        Parameters
        ----------
        x : float
            The point at which to evaluate the CDF.
        
        Returns
        -------
        float
            The value of the CDF.
        """
        return 0.5 * (1.0 + math.erf(x / math.sqrt(2)))

    def simulate_asset_paths(self):
        """
        Simulate asset price paths using geometric Brownian motion.
        
        Returns
        -------
        np.ndarray
            A 2D array of simulated asset prices with shape (paths, steps+1).
        """
        dt = self.T / self.steps
        # Generate random normal increments
        rand_increments = np.random.normal(0, 1, (self.paths, self.steps))
        # Compute log returns
        log_returns = (self.r - 0.5 * self.sigma ** 2) * dt + self.sigma * np.sqrt(dt) * rand_increments
        # Cumulative sum to get log of asset prices; prepend zeros for time 0
        log_paths = np.concatenate(
            [np.zeros((self.paths, 1)), np.cumsum(log_returns, axis=1)], axis=1
        )
        # Convert log prices to actual prices
        paths = self.S0 * np.exp(log_paths)
        return paths

    def european_option_price(self, option_type="put"):
        """
        Calculate the closed-form European option price using the Black–Scholes formula.
        
        Parameters
        ----------
        option_type : str, optional
            'put' or 'call' (default is 'put').
        
        Returns
        -------
        float
            The Black–Scholes price of the European option.
        """
        try:
            d1 = (math.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (
                self.sigma * math.sqrt(self.T)
            )
            d2 = d1 - self.sigma * math.sqrt(self.T)
        except Exception as e:
            raise ValueError("Error computing d1 and d2: " + str(e))
        
        if option_type.lower() == "call":
            price = self.S0 * self._norm_cdf(d1) - self.K * math.exp(-self.r * self.T) * self._norm_cdf(d2)
        elif option_type.lower() == "put":
            price = self.K * math.exp(-self.r * self.T) * self._norm_cdf(-d2) - self.S0 * self._norm_cdf(-d1)
        else:
            raise ValueError("Invalid option type. Must be 'put' or 'call'.")
        return price

    def american_option_price(self, option_type="put"):
        """
        Calculate the American option price using the Longstaff–Schwartz least squares Monte Carlo method.
        
        Parameters
        ----------
        option_type : str, optional
            'put' or 'call' (default is 'put').
        
        Returns
        -------
        float
            The estimated American option price.
        """
        # Simulate asset paths
        paths_array = self.simulate_asset_paths()  # shape: (paths, steps+1)
        dt = self.T / self.steps
        discount = math.exp(-self.r * dt)

        # Compute immediate payoff matrix for each time step and each path
        if option_type.lower() == "put":
            payoff = np.maximum(self.K - paths_array, 0)
        elif option_type.lower() == "call":
            payoff = np.maximum(paths_array - self.K, 0)
        else:
            raise ValueError("Invalid option type. Must be 'put' or 'call'.")

        # Initialize cashflow at maturity (time = T)
        V = payoff[:, -1].copy()

        # Backward induction using Longstaff–Schwartz
        # Iterate from the second-to-last time step down to 1 (no exercise at time 0)
        for t in range(self.steps - 1, 0, -1):
            # Discount the cashflow from time t+1 to time t
            V = discount * V

            # Find indices of paths that are in the money at time t
            in_the_money = payoff[:, t] > 0
            if not np.any(in_the_money):
                continue

            # For regression, use basis functions: 1, S, S^2
            X = paths_array[in_the_money, t]
            A = np.vstack([np.ones_like(X), X, X ** 2]).T

            # Least squares regression to estimate the continuation value
            coeffs = np.linalg.lstsq(A, V[in_the_money], rcond=None)[0]
            continuation_value = coeffs[0] + coeffs[1] * X + coeffs[2] * X ** 2

            # Decide whether to exercise: if immediate payoff exceeds the estimated continuation value, exercise.
            exercise = payoff[in_the_money, t] > continuation_value

            # Get indices (in the original array) corresponding to these in‐the‐money paths
            indices = np.where(in_the_money)[0]
            exercise_indices = indices[exercise]
            V[exercise_indices] = payoff[exercise_indices, t]

        # After looping, V is the discounted payoff at time dt. Discount one more period to get time 0 value.
        american_price = discount * np.mean(V)
        return american_price


# ----------------------- Test Code ----------------------- #
if __name__ == "__main__":
    # Instantiate the option valuation class with default parameters
    option_model = OptionValuation()

    # Calculate European option prices (closed-form)
    euro_put = option_model.european_option_price(option_type="put")
    euro_call = option_model.european_option_price(option_type="call")

    print(f"European Put Price (closed-form): {euro_put:.3f} (Ref: ~3.844)")
    print(f"European Call Price (closed-form): {euro_call:.3f}")

    # Measure execution time for American option valuation
    start_time = time.perf_counter()
    amer_put = option_model.american_option_price(option_type="put")
    amer_call = option_model.american_option_price(option_type="call")
    end_time = time.perf_counter()
    elapsed = end_time - start_time

    print(f"American Put Price (simulation): {amer_put:.3f} (Ref: ~4.478)")
    print(f"American Call Price (simulation): {amer_call:.3f}")
    print(f"Execution Time for American option valuation: {elapsed:.3f} seconds")


## Gemini

### Flash 2.0 (free version)

In [ ]:
import numpy as np
import scipy.stats as si
import time

class BlackScholesOptionPricer:
    """
    Class for valuing American and European options using the Black-Scholes-Merton model.
    """

    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        """
        Initializes the option pricer with given parameters.

        Args:
            S0 (float): Initial stock price.
            K (float): Strike price.
            T (float): Time to maturity (in years).
            r (float): Risk-free interest rate.
            sigma (float): Volatility.
            paths (int): Number of simulation paths.
            steps (int): Number of time steps.
        """

        # Input validation
        if any(not isinstance(x, (int, float)) or x <= 0 for x in [S0, K, T, sigma]):
            raise ValueError("S0, K, T, and sigma must be positive numbers.")
        if not isinstance(r, (int, float)):
             raise ValueError("r must be a number.")
        if not isinstance(paths, int) or paths <= 0:
            raise ValueError("paths must be a positive integer.")
        if not isinstance(steps, int) or steps <= 0:
            raise ValueError("steps must be a positive integer.")


        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps
        self.dt = T / steps

    def value_american_put(self):
        """
        Values an American put option using the Longstaff-Schwartz method.

        Returns:
            float: American put option value.
        """
        S = np.zeros((self.paths, self.steps + 1))
        S[:, 0] = self.S0

        for i in range(self.steps):
            S[:, i + 1] = S[:, i] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt +
                                           self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.paths))

        V = np.maximum(self.K - S[:, -1], 0)  # Intrinsic value at maturity
        for i in range(self.steps - 1, 0, -1):
            intrinsic_value = np.maximum(self.K - S[:, i], 0)
            # Regression: Fit a polynomial to estimate continuation value
            X = S[:, i]
            Y = V * np.exp(-self.r * self.dt)  # Discounted future cash flows
            try:
                coefficients = np.polyfit(X, Y, 2)  # Quadratic regression
            except np.linalg.LinAlgError:  # Handle cases where polyfit may fail
                coefficients = np.polyfit(X, Y, 1) # Fallback to linear
            continuation_value = np.polyval(coefficients, X) # Calculate continuation value

            V = np.where(intrinsic_value > continuation_value, intrinsic_value, V * np.exp(-self.r * self.dt))

        return np.mean(V) * np.exp(-self.r * self.dt)

    def value_european_put(self):
        """
        Values a European put option using the Black-Scholes formula.

        Returns:
            float: European put option value.
        """
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return self.K * np.exp(-self.r * self.T) * si.norm.cdf(-d2) - self.S0 * si.norm.cdf(-d1)

    def value_american_call(self):  # Added American Call Option
        """
        Values an American call option using the Longstaff-Schwartz method.

        Returns:
            float: American call option value.
        """
        S = np.zeros((self.paths, self.steps + 1))
        S[:, 0] = self.S0

        for i in range(self.steps):
            S[:, i + 1] = S[:, i] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt +
                                           self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.paths))

        V = np.maximum(S[:, -1] - self.K, 0)  # Intrinsic value at maturity
        for i in range(self.steps - 1, 0, -1):
            intrinsic_value = np.maximum(S[:, i] - self.K, 0)
            # Regression: Fit a polynomial to estimate continuation value
            X = S[:, i]
            Y = V * np.exp(-self.r * self.dt)  # Discounted future cash flows
            try:
                coefficients = np.polyfit(X, Y, 2)  # Quadratic regression
            except np.linalg.LinAlgError:  # Handle cases where polyfit may fail
                coefficients = np.polyfit(X, Y, 1) # Fallback to linear
            continuation_value = np.polyval(coefficients, X)  # Calculate continuation value

            V = np.where(intrinsic_value > continuation_value, intrinsic_value, V * np.exp(-self.r * self.dt))

        return np.mean(V) * np.exp(-self.r * self.dt)


    def value_european_call(self): # Added European call option
        """
        Values a European call option using the Black-Scholes formula.

        Returns:
            float: European call option value.
        """
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return self.S0 * si.norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * si.norm.cdf(d2)



# Test code
pricer = BlackScholesOptionPricer()

start_time = time.time()
american_put_value = pricer.value_american_put()
american_call_value = pricer.value_american_call() # Added American Call Value calculation
end_time = time.time()

european_put_value = pricer.value_european_put()
european_call_value = pricer.value_european_call() # Added European Call Value calculation

print(f"American Put Value: {american_put_value}")
print(f"American Call Value: {american_call_value}") # Print American Call Value
print(f"European Put Value: {european_put_value}")
print(f"European Call Value: {european_call_value}") # Print European Call Value
print(f"American Valuation Time: {end_time - start_time:.4f} seconds")

### Flash 2.0 (Advanced Gemini = paid version)

In [ ]:
import numpy as np
import scipy.stats as si
import time

class BlackScholesMerton:
    """
    Class for valuing American and European options using the Black-Scholes-Merton model.
    """

    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        """
        Initializes the option parameters.

        Args:
            S0 (float): Initial stock price.
            K (float): Strike price.
            T (float): Time to maturity.
            r (float): Risk-free interest rate.
            sigma (float): Volatility.
            paths (int): Number of simulation paths.
            steps (int): Number of time steps.
        """

        # Input validation
        if not all(isinstance(x, (int, float)) and x > 0 for x in [S0, K, T, r, sigma]) or paths <= 0 or steps <= 0:
            raise ValueError("Invalid input parameters. All numerical values must be positive.")

        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps
        self.dt = T / steps

    def value_american_put(self):
        """
        Values an American put option using the Longstaff-Schwartz (2001) method.

        Returns:
            float: American put option value.
        """
        try:
          # Generate stock price paths
          Z = np.random.normal(size=(self.steps + 1, self.paths))
          ST = np.zeros_like(Z)
          ST[0] = self.S0
          for i in range(1, self.steps + 1):
              ST[i] = ST[i - 1] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt + self.sigma * np.sqrt(self.dt) * Z[i])

          # Backward induction
          V = np.maximum(self.K - ST[-1], 0)  # Intrinsic value at maturity
          for t in range(self.steps - 1, -1, -1):
              # Regression (using least squares)
              X = ST[t]
              Y = V * np.exp(-self.r * self.dt) # Discounted continuation value
              if t>0: #Avoids issues with polynomial regression on the first step
                  coefficients = np.polyfit(X, Y, 2) # Fit a polynomial
                  EV = np.polyval(coefficients, X) # Estimated continuation value
              else:
                  EV = Y
              V = np.maximum(self.K - ST[t], EV)  # Exercise or continue

          return np.exp(-self.r * self.T) * np.mean(V[0]) # Discounted average payoff

        except Exception as e:
            raise RuntimeError(f"Error in American put valuation: {e}")

    def value_european_put(self):
        """
        Values a European put option using the Black-Scholes formula.

        Returns:
            float: European put option value.
        """
        try:
            d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
            d2 = d1 - self.sigma * np.sqrt(self.T)
            return self.K * np.exp(-self.r * self.T) * si.norm.cdf(-d2) - self.S0 * si.norm.cdf(-d1)
        except Exception as e:
            raise RuntimeError(f"Error in European put valuation: {e}")

    def value_american_call(self):
        """
        Values an American call option using the Longstaff-Schwartz (2001) method.

        Returns:
            float: American call option value.
        """
        try:
            # Generate stock price paths (same as for put)
            Z = np.random.normal(size=(self.steps + 1, self.paths))
            ST = np.zeros_like(Z)
            ST[0] = self.S0
            for i in range(1, self.steps + 1):
                ST[i] = ST[i - 1] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt + self.sigma * np.sqrt(self.dt) * Z[i])

            # Backward induction (similar to put, but with call payoff)
            V = np.maximum(ST[-1] - self.K, 0)  # Intrinsic value at maturity
            for t in range(self.steps - 1, -1, -1):
                X = ST[t]
                Y = V * np.exp(-self.r * self.dt)
                if t>0:
                    coefficients = np.polyfit(X, Y, 2)
                    EV = np.polyval(coefficients, X)
                else:
                    EV = Y

                V = np.maximum(ST[t] - self.K, EV)
            return np.exp(-self.r * self.T) * np.mean(V[0])

        except Exception as e:
            raise RuntimeError(f"Error in American call valuation: {e}")


    def value_european_call(self):
        """
        Values a European call option using the Black-Scholes formula.

        Returns:
            float: European call option value.
        """
        try:
            d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
            d2 = d1 - self.sigma * np.sqrt(self.T)
            return self.S0 * si.norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * si.norm.cdf(d2)
        except Exception as e:
            raise RuntimeError(f"Error in European call valuation: {e}")



# Test code
if __name__ == "__main__":
    option = BlackScholesMerton()

    # Calculate option values
    american_put = option.value_american_put()
    european_put = option.value_european_put()
    american_call = option.value_american_call()
    european_call = option.value_european_call()


    print(f"American Put Value: {american_put}")
    print(f"European Put Value: {european_put}")
    print(f"American Call Value: {american_call}")
    print(f"European Call Value: {european_call}")


    # Measure execution time for American put valuation
    start_time = time.time()
    american_call = option.value_american_call()
    american_put = option.value_american_put()  # Recalculate for timing
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"American Put Valuation Time: {execution_time:.4f} seconds")

**_COMMENT: The code does not value the American (put) option correctly. There is great variability in the option values (both on the high and the low side)._**

## Claude

In [ ]:
import numpy as np
import time
from scipy.stats import norm


class OptionPricing:
    """
    A class for valuing American and European options using the Black-Scholes-Merton model.
    For American options, the Longstaff-Schwartz (2001) least squares Monte Carlo method is implemented.
    For European options, the closed-form Black-Scholes formulas are used.
    """

    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        """
        Initialize the option pricing model with parameters.
        
        Parameters:
        -----------
        S0 : float
            Initial stock price
        K : float
            Strike price
        T : float
            Time to maturity in years
        r : float
            Risk-free interest rate (annualized)
        sigma : float
            Volatility of the underlying asset (annualized)
        paths : int
            Number of simulation paths
        steps : int
            Number of time steps in the simulation
        """
        self._validate_inputs(S0, K, T, r, sigma, paths, steps)
        
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps
        self.dt = T / steps
        
    def _validate_inputs(self, S0, K, T, r, sigma, paths, steps):
        """Validate input parameters to ensure they are within reasonable ranges."""
        if S0 <= 0:
            raise ValueError("Initial stock price (S0) must be positive")
        if K <= 0:
            raise ValueError("Strike price (K) must be positive")
        if T <= 0:
            raise ValueError("Time to maturity (T) must be positive")
        if sigma <= 0:
            raise ValueError("Volatility (sigma) must be positive")
        if paths <= 0 or not isinstance(paths, int):
            raise ValueError("Number of paths must be a positive integer")
        if steps <= 0 or not isinstance(steps, int):
            raise ValueError("Number of steps must be a positive integer")
            
    def generate_paths(self):
        """
        Generate stock price paths using geometric Brownian motion.
        
        Returns:
        --------
        numpy.ndarray
            Array of simulated stock price paths with shape (paths, steps+1)
        """
        # Initialize the stock price array
        stock_prices = np.zeros((self.paths, self.steps + 1))
        stock_prices[:, 0] = self.S0
        
        # Generate random normal variables for the simulation
        Z = np.random.standard_normal((self.paths, self.steps))
        
        # Calculate the drift and diffusion terms
        drift = (self.r - 0.5 * self.sigma ** 2) * self.dt
        diffusion = self.sigma * np.sqrt(self.dt)
        
        # Generate the paths using vectorized operations
        for t in range(1, self.steps + 1):
            stock_prices[:, t] = stock_prices[:, t-1] * np.exp(drift + diffusion * Z[:, t-1])
            
        return stock_prices
        
    def price_european_option_closed_form(self, option_type='put'):
        """
        Calculate European option price using the Black-Scholes closed-form solution.
        
        Parameters:
        -----------
        option_type : str
            Type of option: 'call' or 'put'
            
        Returns:
        --------
        float
            The price of the European option
        """
        if option_type not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'")
            
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        
        if option_type == 'call':
            price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:  # put option
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)
            
        return price
        
    def price_american_option(self, option_type='put'):
        """
        Price American options using the Longstaff-Schwartz least squares Monte Carlo method.
        
        Parameters:
        -----------
        option_type : str
            Type of option: 'call' or 'put'
            
        Returns:
        --------
        float
            The price of the American option
        """
        if option_type not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'")
            
        # Generate stock price paths
        paths = self.generate_paths()
        
        # Initialize cash flow matrix
        cash_flows = np.zeros_like(paths)
        
        # Calculate the discount factor for each time step
        discount_factors = np.exp(-self.r * self.dt * np.arange(self.steps + 1))
        
        # Calculate the intrinsic values at expiration (time step = steps)
        if option_type == 'call':
            cash_flows[:, -1] = np.maximum(0, paths[:, -1] - self.K)
        else:  # put option
            cash_flows[:, -1] = np.maximum(0, self.K - paths[:, -1])
            
        # Backward induction through the tree
        for t in range(self.steps - 1, 0, -1):
            # Calculate the present value of future cash flows
            present_value = cash_flows[:, t+1:] * discount_factors[1:(self.steps+1-t)]
            present_value = np.sum(present_value, axis=1)
            
            # Calculate the intrinsic value at the current time step
            if option_type == 'call':
                intrinsic_value = np.maximum(0, paths[:, t] - self.K)
            else:  # put option
                intrinsic_value = np.maximum(0, self.K - paths[:, t])
                
            # Identify in-the-money paths
            itm_indices = np.where(intrinsic_value > 0)[0]
            
            if len(itm_indices) > 0:
                # Get the stock prices and present values for in-the-money paths
                itm_stock_prices = paths[itm_indices, t]
                itm_present_values = present_value[itm_indices]
                
                # Create basis functions (polynomial terms) for regression
                X = np.column_stack([
                    np.ones(len(itm_indices)),
                    itm_stock_prices,
                    itm_stock_prices ** 2,
                    itm_stock_prices ** 3
                ])
                
                # Perform least-squares regression
                try:
                    beta, _, _, _ = np.linalg.lstsq(X, itm_present_values, rcond=None)
                    
                    # Calculate the continuation value (expected future cash flows)
                    continuation_value = np.dot(X, beta)
                    
                    # Compare intrinsic value and continuation value to make exercise decision
                    itm_intrinsic_value = intrinsic_value[itm_indices]
                    exercise = itm_intrinsic_value > continuation_value
                    
                    # Update cash flows based on exercise decision
                    cash_flows[itm_indices, t] = np.where(
                        exercise,
                        itm_intrinsic_value,
                        0
                    )
                    
                    # If exercised, set future cash flows to 0
                    for i, idx in enumerate(itm_indices):
                        if exercise[i]:
                            cash_flows[idx, t+1:] = 0
                except np.linalg.LinAlgError:
                    # Fallback in case of numerical issues
                    pass
            
        # Calculate the intrinsic value at t=0 (no need for regression)
        if option_type == 'call':
            intrinsic_value_t0 = max(0, self.S0 - self.K)
        else:  # put option
            intrinsic_value_t0 = max(0, self.K - self.S0)
        
        # Calculate the present value of all future cash flows at t=0
        future_cash_flows = np.sum(cash_flows * discount_factors.reshape(1, -1), axis=1)
        continuation_value_t0 = np.mean(future_cash_flows)
        
        # The American option value is the maximum of immediate exercise and continuation
        return max(intrinsic_value_t0, continuation_value_t0)
        
    def price_european_option_simulation(self, option_type='put'):
        """
        Price European options using Monte Carlo simulation for verification purposes.
        
        Parameters:
        -----------
        option_type : str
            Type of option: 'call' or 'put'
            
        Returns:
        --------
        float
            The price of the European option using simulation
        """
        if option_type not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'")
            
        # Generate stock price paths
        paths = self.generate_paths()
        
        # Calculate payoffs at maturity
        if option_type == 'call':
            payoffs = np.maximum(0, paths[:, -1] - self.K)
        else:  # put option
            payoffs = np.maximum(0, self.K - paths[:, -1])
            
        # Discount payoffs to present value
        option_price = np.exp(-self.r * self.T) * np.mean(payoffs)
        
        return option_price


def run_tests():
    """Run tests comparing American and European option prices with reference values"""
    print("Option Pricing Test")
    print("-" * 50)
    
    # Create option pricing model with default values
    model = OptionPricing()
    
    # Calculate European put option price (closed-form)
    european_put_price = model.price_european_option_closed_form(option_type='put')
    european_call_price = model.price_european_option_closed_form(option_type='call')
    
    # Time the American put option pricing
    start_time = time.time()
    american_put_price = model.price_american_option(option_type='put')
    american_put_time = time.time() - start_time
    
    # Calculate American call option price
    start_time = time.time()
    american_call_price = model.price_american_option(option_type='call')
    american_call_time = time.time() - start_time
    
    # Print results
    print(f"Model Parameters:")
    print(f"  S0 = {model.S0}, K = {model.K}, T = {model.T}")
    print(f"  r = {model.r}, sigma = {model.sigma}")
    print(f"  paths = {model.paths}, steps = {model.steps}")
    print("\nOption Prices:")
    print(f"  American Put: ${american_put_price:.3f} (Ref: $4.478) - Computed in {american_put_time:.3f} seconds")
    print(f"  European Put: ${european_put_price:.3f} (Ref: $3.844)")
    print(f"  American Call: ${american_call_price:.3f} - Computed in {american_call_time:.3f} seconds")
    print(f"  European Call: ${european_call_price:.3f}")
    
    # Calculate early exercise premium
    put_premium = american_put_price - european_put_price
    call_premium = american_call_price - european_call_price
    
    print("\nEarly Exercise Premium:")
    print(f"  Put Premium: ${put_premium:.3f}")
    print(f"  Call Premium: ${call_premium:.3f}")


if __name__ == "__main__":
    run_tests()

## DeepSeek

### v3

In [ ]:
import numpy as np
from scipy.stats import norm
from time import time

class OptionPricer:
    """
    A class to value American and European options using the Black-Scholes-Merton model.
    For American options, the Longstaff-Schwartz least squares Monte Carlo method is used.
    For European options, the closed-form Black-Scholes formula is used.
    """

    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        """
        Initialize the OptionPricer with default or user-provided parameters.
        
        Parameters:
        S0 (float): Initial stock price. Default is 36.
        K (float): Strike price. Default is 40.
        T (float): Time to maturity in years. Default is 1.0.
        r (float): Risk-free interest rate. Default is 0.06.
        sigma (float): Volatility of the underlying asset. Default is 0.2.
        paths (int): Number of Monte Carlo simulation paths. Default is 100000.
        steps (int): Number of time steps in the simulation. Default is 50.
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps

        # Validate inputs
        self._validate_inputs()

    def _validate_inputs(self):
        """Validate input parameters to ensure they are valid."""
        if self.S0 <= 0 or self.K <= 0 or self.T <= 0 or self.sigma <= 0 or self.paths <= 0 or self.steps <= 0:
            raise ValueError("All input parameters must be positive.")
        if self.r < 0:
            raise ValueError("Risk-free rate cannot be negative.")

    def _black_scholes_price(self, option_type):
        """
        Calculate the European option price using the Black-Scholes formula.
        
        Parameters:
        option_type (str): 'call' or 'put'.
        
        Returns:
        float: European option price.
        """
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)

        if option_type == 'call':
            price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        elif option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)
        else:
            raise ValueError("Invalid option type. Use 'call' or 'put'.")

        return price

    def _monte_carlo_simulation(self):
        """
        Simulate stock price paths using geometric Brownian motion.
        
        Returns:
        numpy.ndarray: Simulated stock price paths.
        """
        dt = self.T / self.steps
        drift = (self.r - 0.5 * self.sigma**2) * dt
        diffusion = self.sigma * np.sqrt(dt)

        # Generate random numbers for the simulation
        rand = np.random.normal(size=(self.paths, self.steps))

        # Simulate stock price paths
        price_paths = np.zeros((self.paths, self.steps + 1))
        price_paths[:, 0] = self.S0

        for t in range(1, self.steps + 1):
            price_paths[:, t] = price_paths[:, t - 1] * np.exp(drift + diffusion * rand[:, t - 1])

        return price_paths

    def _longstaff_schwartz(self, price_paths, option_type):
        """
        Value American options using the Longstaff-Schwartz least squares Monte Carlo method.
        
        Parameters:
        price_paths (numpy.ndarray): Simulated stock price paths.
        option_type (str): 'call' or 'put'.
        
        Returns:
        float: American option price.
        """
        dt = self.T / self.steps
        discount = np.exp(-self.r * dt)

        # Initialize the payoff matrix
        payoff = np.maximum(self.K - price_paths, 0) if option_type == 'put' else np.maximum(price_paths - self.K, 0)
        value = payoff[:, -1]

        # Backward induction
        for t in range(self.steps - 1, 0, -1):
            # In-the-money paths
            in_the_money = payoff[:, t] > 0
            X = price_paths[in_the_money, t]
            Y = value[in_the_money] * discount

            # Regression to estimate continuation value
            if X.size > 0:
                A = np.vstack([X, X**2, np.ones_like(X)]).T
                coeffs = np.linalg.lstsq(A, Y, rcond=None)[0]
                continuation_value = np.dot(A, coeffs)

                # Update value for in-the-money paths
                exercise_value = payoff[in_the_money, t]
                value[in_the_money] = np.where(exercise_value > continuation_value, exercise_value, Y)

        # Discount back to present
        option_price = np.mean(value) * discount
        return option_price

    def calculate_european_option_price(self, option_type):
        """
        Calculate the European option price using the Black-Scholes formula.
        
        Parameters:
        option_type (str): 'call' or 'put'.
        
        Returns:
        float: European option price.
        """
        return self._black_scholes_price(option_type)

    def calculate_american_option_price(self, option_type):
        """
        Calculate the American option price using the Longstaff-Schwartz method.
        
        Parameters:
        option_type (str): 'call' or 'put'.
        
        Returns:
        float: American option price.
        """
        price_paths = self._monte_carlo_simulation()
        return self._longstaff_schwartz(price_paths, option_type)


# Test code
if __name__ == "__main__":
    # Initialize the OptionPricer
    pricer = OptionPricer()

    # Calculate European option prices
    european_put_price = pricer.calculate_european_option_price('put')
    european_call_price = pricer.calculate_european_option_price('call')

    # Calculate American option prices and measure execution time
    start_time = time()
    american_put_price = pricer.calculate_american_option_price('put')
    american_call_price = pricer.calculate_american_option_price('call')
    execution_time = time() - start_time

    # Print results
    print(f"European Put Option Price: {european_put_price:.3f}")
    print(f"European Call Option Price: {european_call_price:.3f}")
    print(f"American Put Option Price: {american_put_price:.3f}")
    print(f"American Call Option Price: {american_call_price:.3f}")
    print(f"Execution Time for American Option Valuation: {execution_time:.2f} seconds")

### R1

In [ ]:
import numpy as np
from scipy.stats import norm
import time

class BSMertonLSM:
    """
    A class to value American and European options using the Black-Scholes-Merton model.
    American options are valued using the Longstaff-Schwartz least squares Monte Carlo method.
    European options are valued using closed-form formulas.
    
    Parameters:
    S0 (float): Initial stock price. Default is 36.
    K (float): Strike price. Default is 40.
    T (float): Time to maturity in years. Default is 1.0.
    r (float): Risk-free interest rate. Default is 0.06.
    sigma (float): Volatility of the underlying asset. Default is 0.2.
    paths (int): Number of Monte Carlo paths. Default is 100000.
    steps (int): Number of time steps for the simulation. Default is 50.
    """
    
    def __init__(self, S0=36, K=40, T=1.0, r=0.06, sigma=0.2, paths=100000, steps=50):
        # Input validation
        if not all(isinstance(arg, (int, float)) for arg in [S0, K, T, r, sigma]):
            raise TypeError("S0, K, T, r, sigma must be numeric")
        if not all(arg > 0 for arg in [S0, K, T, r, sigma]):
            raise ValueError("S0, K, T, r, sigma must be positive")
        if not isinstance(paths, int) or paths <= 0:
            raise ValueError("paths must be a positive integer")
        if not isinstance(steps, int) or steps <= 0:
            raise ValueError("steps must be a positive integer")
        
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.paths = paths
        self.steps = steps

    def _d1(self, S0, K, T, r, sigma):
        return (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    def _d2(self, S0, K, T, r, sigma):
        return self._d1(S0, K, T, r, sigma) - sigma * np.sqrt(T)
    
    def european_call(self):
        """Calculate the European call option price using the Black-Scholes formula."""
        d1 = self._d1(self.S0, self.K, self.T, self.r, self.sigma)
        d2 = self._d2(self.S0, self.K, self.T, self.r, self.sigma)
        call_price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        return call_price
    
    def european_put(self):
        """Calculate the European put option price using the Black-Scholes formula."""
        d1 = self._d1(self.S0, self.K, self.T, self.r, self.sigma)
        d2 = self._d2(self.S0, self.K, self.T, self.r, self.sigma)
        put_price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)
        return put_price
    
    def american_option(self, option_type='put'):
        """
        Value an American option using the Longstaff-Schwartz least squares Monte Carlo method.
        
        Parameters:
        option_type (str): 'put' or 'call'. Default is 'put'.
        
        Returns:
        float: Estimated American option value.
        """
        if option_type not in ['put', 'call']:
            raise ValueError("option_type must be 'put' or 'call'")
        
        dt = self.T / self.steps
        drift = (self.r - 0.5 * self.sigma**2) * dt
        vol = self.sigma * np.sqrt(dt)
        
        # Generate random numbers and stock paths
        Z = np.random.standard_normal((self.steps, self.paths))
        log_returns = drift + vol * Z
        log_returns = np.vstack([np.zeros(self.paths), log_returns])  # Include time 0
        
        cumulative_returns = np.cumsum(log_returns, axis=0)
        S = self.S0 * np.exp(cumulative_returns)
        
        cash_flows = np.zeros_like(S)
        if option_type == 'put':
            cash_flows[-1] = np.maximum(self.K - S[-1], 0.0)
        else:
            cash_flows[-1] = np.maximum(S[-1] - self.K, 0.0)
        
        exercise = np.zeros(self.paths, dtype=bool)
        Y_prev = np.zeros(self.paths)
        
        for t in reversed(range(self.steps)):  # t is the current step index (0 to steps-1)
            current_S = S[t]
            
            # Determine in-the-money paths
            if option_type == 'put':
                in_money = current_S < self.K
            else:
                in_money = current_S > self.K
            
            candidates = in_money & ~exercise
            
            if not np.any(candidates):
                Y_current = (cash_flows[t+1] + Y_prev) * np.exp(-self.r * dt)
                Y_prev = Y_current
                continue
            
            X = current_S[candidates]
            Y_t_candidates = (cash_flows[t+1][candidates] + Y_prev[candidates]) * np.exp(-self.r * dt)
            
            # Basis functions: 1, X, X^2
            X_basis = np.column_stack([np.ones_like(X), X, X**2])
            beta = np.linalg.lstsq(X_basis, Y_t_candidates, rcond=None)[0]
            continuation = X_basis @ beta
            
            if option_type == 'put':
                exercise_values = self.K - X
            else:
                exercise_values = X - self.K
            
            exercise_now = exercise_values > continuation
            
            # Update cash flows and exercise indicators
            cash_flows[t, candidates] = np.where(exercise_now, exercise_values, cash_flows[t, candidates])
            new_exercised = np.zeros_like(exercise)
            new_exercised[candidates] = exercise_now
            exercise |= new_exercised
            
            # Update Y_prev for next iteration
            Y_current = (cash_flows[t+1] + Y_prev) * np.exp(-self.r * dt)
            Y_current[new_exercised] = 0.0
            Y_prev = Y_current
        
        # Calculate the option value
        mask = cash_flows > 0
        has_exercise = mask.any(axis=0)
        exercise_time = np.argmax(mask, axis=0)
        cash_flow_values = cash_flows[exercise_time, np.arange(self.paths)]
        discount_factors = np.exp(-self.r * dt * exercise_time)
        discounted_values = cash_flow_values * discount_factors * has_exercise
        return np.mean(discounted_values)

if __name__ == '__main__':
    # Initialize the model with default parameters
    bsm = BSMertonLSM()
    
    # Calculate European option prices
    european_put = bsm.european_put()
    european_call = bsm.european_call()
    
    # Calculate American option prices and measure time
    start_put = time.time()
    american_put = bsm.american_option('put')
    time_put = time.time() - start_put
    
    start_call = time.time()
    american_call = bsm.american_option('call')
    time_call = time.time() - start_call
    
    # Print results
    print(f"European Put Option: {european_put:.3f}")
    print(f"European Call Option: {european_call:.3f}")
    print(f"American Put Option: {american_put:.3f} (execution time: {time_put:.2f} seconds)")
    print(f"American Call Option: {american_call:.3f} (execution time: {time_call:.2f} seconds)")

**_COMMENT: The only result that included a syntax error! (although an easy to fix one). Accuracy of the American put option values also not high enough._**

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>